<a href="https://colab.research.google.com/github/myDSMLProjects/PyTorch-Fundamentals/blob/master/PyTorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

In [ ]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)  # (B, 1, 28, 28)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc1 = nn.Linear(in_features = 16*7*7, out_features=num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)

    return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyperparameters
learning_rate=0.001
batch_size=64
num_epochs = 5

In [ ]:
train_dataset = datasets.MNIST(root = 'datasets/MNIST', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='datasets/MNIST', download=True, train=False, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [ ]:
# Creating the model and initializing the network
model = CNN()
model.to(device)

# define loss criterion and optimizer
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lr = learning_rate, params = model.parameters())

for epoch in range(num_epochs):
  for batch_idx, (data,targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)

    scores = model(data)
    loss = loss_criterion(scores, targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

  def check_accuracy(loader, model):
    if loader.dataset.train:
      print('Checking accuracy on training data')
    else:
      print('Checking accuracy on test data')

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
      for x, y in loader:
        x = x.to(device)
        y = y.to(device)

        scores = model(x)
        _, predictions = scores.max(1)
        num_correct += (predictions==y).sum()
        num_samples += predictions.size(0)
      
      print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

  check_accuracy(train_loader, model)
  check_accuracy(test_loader, model)

Checking accuracy on training data
Got 57888/60000 with accuracy 96.48
Checking accuracy on test data
Got 9665/10000 with accuracy 96.65
Checking accuracy on training data
Got 58662/60000 with accuracy 97.77
Checking accuracy on test data
Got 9781/10000 with accuracy 97.81
Checking accuracy on training data
Got 58910/60000 with accuracy 98.18
Checking accuracy on test data
Got 9799/10000 with accuracy 97.99
Checking accuracy on training data
Got 59088/60000 with accuracy 98.48
Checking accuracy on test data
Got 9837/10000 with accuracy 98.37
Checking accuracy on training data
Got 59184/60000 with accuracy 98.64
Checking accuracy on test data
Got 9854/10000 with accuracy 98.54
